##**Steps in the Code:**

1. Define the LeNet class: Implements the CNN architecture.
2. Train and save the model: Use MNIST dataset.
3. Load the saved model: Load the trained model from the saved file.
4. Test the model: Use hand-written digit images for predictions.



##**Import required libraries -**

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os

##**Define the LeNet class -**

In [ ]:
class LeNet:
    def __init__(self):
      """
        Initializes the LeNet CNN architecture with layers for image classification.
      """
        self.model = models.Sequential([
            layers.Conv2D(6, kernel_size=(5, 5), activation='relu', input_shape=(28, 28, 1)),
            layers.AveragePooling2D(pool_size=(2, 2)),
            layers.Conv2D(16, kernel_size=(5, 5), activation='relu'),
            layers.AveragePooling2D(pool_size=(2, 2)),
            layers.Flatten(),
            layers.Dense(120, activation='relu'),
            layers.Dense(84, activation='relu'),
            layers.Dense(10, activation='softmax')
        ])
        self.model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    def train(self, x_train, y_train, x_test, y_test, epochs=5):
      """
        Trains the model on the provided dataset.
        Args:
            x_train (numpy.ndarray): Training data.
            y_train (numpy.ndarray): Labels for the training data.
            x_test (numpy.ndarray): Test data.
            y_test (numpy.ndarray): Labels for the test data.
            epochs (int): Number of training epochs.
        Returns:
            tensorflow.keras.Model: The trained model.
      """
        self.model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=epochs)
        return self.model

    def save(self, path='utekar_cnn_model.keras'):
      """
        Saves the trained model to the specified path.
        Args:
            path (str): File path to save the model.
      """
        self.model.save(path)

    @staticmethod
    def load(path='utekar_cnn_model.keras'):
      """
        Loads a saved model from the specified path.
        Args:
            path (str): File path of the saved model.
        Returns:
            tensorflow.keras.Model: The loaded model.
      """
        return tf.keras.models.load_model(path)

##**Load and preprocess the MNIST dataset -**

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype("float32") / 255.0

##**Train, Save, Load & Test the model handwritten digit images -**

In [ ]:
lenet = LeNet()
lenet.train(x_train, y_train, x_test, y_test, epochs=5)
lenet.save()

loaded_model = LeNet.load()

def load_image(image_path):
    """Loads and preprocesses an image."""
    image = Image.open(image_path).convert('L')  # Convert to grayscale
    image = image.resize((28, 28))  # Resize to match MNIST images
    image = np.array(image).astype("float32") / 255.0
    image = np.expand_dims(image, axis=(0, -1))  # Add batch and channel dimensions
    return image

def test_model(image_path, actual_digit):
    """Tests the model with a single image."""
    image = load_image(image_path)
    prediction = np.argmax(loaded_model.predict(image))
    plt.imshow(image.squeeze(), cmap='gray')
    plt.title(f"Actual: {actual_digit}, Predicted: {prediction}")
    plt.axis("off")
    plt.show()
    if prediction == actual_digit:
        print(f"Success: Model correctly predicted {prediction}.")
    else:
        print(f"Failure: Model predicted {prediction}, but actual digit is {actual_digit}.")

# Test handwritten digit images
test_images_path = "/content/handwritten_digits"  # Update to your images folder
for img_file in os.listdir(test_images_path):
    if img_file.endswith('.png') or img_file.endswith('.jpg'):
        actual_digit = int(img_file.split('_')[0])  # Assuming filenames like '3_image.png'
        test_model(os.path.join(test_images_path, img_file), actual_digit)